# Settings

In [1]:
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [2]:
!pip install ratsnlp

In [3]:
import torch
from ratsnlp.nlpbook.qa import QATrainArguments
args = QATrainArguments(
    pretrained_model_name='beomi/kcbert-base',
    downstream_corpus_name='korquad-v1',
    downstream_corpus_root_dir='.data/Korpora',
    downstream_model_dir='.checkpoint-qa',
    max_seq_length=128, # 입력 문장 최대 길이(질문과 지문 모두 포함)
    max_query_length=32, # 질문 최대 길이
    doc_stride=64, # 지문에서 몇 개 토큰을 슬라이딩해가면서 데이터를 늘릴지 결정
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=52-5,
    epochs=3,
    tpu_cores = 0 if torch.cuda.is_available() else 8,
    seed=7,
)

In [4]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


# 말뭉치 다운로드

In [5]:
nlpbook.download_downstream_dataset(args)

# 토크나이저 설정

In [6]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

# 데이터 전처리

In [7]:
from ratsnlp.nlpbook.qa import KorQuADV1Corpus, QADataset
corpus = KorQuADV1Corpus()
train_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode='train',
)

In [8]:
train_dataset[0]

QAFeatures(input_ids=[2, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 19143, 13985, 12449, 9194, 4105, 3385, 9411, 32, 3, 8601, 4633, 29697, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 4104, 4027, 8793, 13985, 391, 9132, 4113, 10966, 11728, 12023, 14657, 4091, 8598, 16639, 341, 4573, 4771, 4027, 2139, 8478, 14416, 214, 8202, 17, 2451, 13007, 1480, 4313, 4538, 4008, 8601, 4633, 22903, 4113, 1676, 868, 4913, 7965, 1789, 4203, 4110, 15031, 786, 250, 4057, 10878, 4007, 2593, 4094, 4128, 10289, 4113, 10958, 4062, 9511, 1355, 4600, 4103, 4775, 5602, 10770, 4180, 26732, 3231, 23243, 4104, 4042, 2015, 4012, 4113, 9198, 8763, 8129, 17, 10384, 23008, 7971, 2170, 4408, 4011, 4147, 4042, 17015, 4091, 23008, 21056, 4165, 323, 4175, 4158, 11413, 2273, 4043, 7966, 1543, 4775, 3], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [9]:
train_dataset[1]

QAFeatures(input_ids=[2, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 19143, 13985, 12449, 9194, 4105, 3385, 9411, 32, 3, 10958, 4062, 9511, 1355, 4600, 4103, 4775, 5602, 10770, 4180, 26732, 3231, 23243, 4104, 4042, 2015, 4012, 4113, 9198, 8763, 8129, 17, 10384, 23008, 7971, 2170, 4408, 4011, 4147, 4042, 17015, 4091, 23008, 21056, 4165, 323, 4175, 4158, 11413, 2273, 4043, 7966, 1543, 4775, 4170, 4042, 341, 4573, 4771, 28, 4566, 4027, 10599, 18907, 208, 9504, 24835, 15, 11060, 2451, 4780, 4032, 18548, 4113, 3231, 23243, 4104, 4042, 1843, 4771, 7965, 28987, 4153, 2451, 15489, 4113, 13928, 17283, 575, 4261, 26783, 8114, 8852, 9107, 4082, 28498, 8131, 17, 8225, 4042, 1114, 4281, 4194, 17138, 4042, 9961, 8222, 14041, 10892, 4113, 2524, 4443, 8032, 12710, 21602, 18625, 24569, 4136, 3], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

# 데이터 로더 구축

In [10]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

In [11]:
from torch.utils.data import SequentialSampler
val_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode='val',
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

# 모델 로드

In [12]:
from transformers import BertConfig, BertForQuestionAnswering
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)
model = BertForQuestionAnswering.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model c

# 모델 학습

In [13]:
from ratsnlp.nlpbook.qa import QATask
task = QATask(model, args)

In [14]:
trainer = nlpbook.get_trainer(args)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [15]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

/home/sbumseo/git/TIL/.venv/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /home/sbumseo/git/TIL/NLP/.checkpoint-qa exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/sbumseo/git/TIL/.venv/lib/python3.8/site-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(

  | Name  | Type                     | Params
---------------------------------------------------
0 | model | BertForQuestionAnswering | 108 M 
---------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.318   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

# 모델 출력 및 후처리

In [19]:
from ratsnlp.nlpbook.qa import QADeployArguments
dargs = QADeployArguments(
    pretrained_model_name='beomi/kcbert-base',
    downstream_model_dir='.checkpoint-qa',
    max_seq_length=128,
    max_query_length=32,
)

downstream_model_checkpoint_fpath: .checkpoint-qa/epoch=1-val_loss=4.85.ckpt


In [20]:
def inference_fn(question, context):
    if question and context:
        # question 토큰화 및 인덱싱
        truncated_query = tokenizer.encode(
            question,
            add_special_tokens=False,
            truncation=True,
            max_length=dargs.max_query_length, # max_length 초과 시 자름
        )
        # truncated_query를 context와 함께 토큰화 및 인덱싱
        inputs = tokenizer.encode_plus(
            text=truncated_query,
            text_pair=context,
            truncation='only_second', # 전체 길이가 max_length 초과 시 자름
            padding='max_length',
            max_length=dargs.max_seq_length,
            return_token_type_ids=True,
        )
        with torch.no_grad():
            outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})

            # 정답의 시작 위치와 관련된 로짓(outputs.start_logits)에서 가장 큰 값이 가리키는 토큰 위치
            start_pred = outputs.start_logits.argmax(dim=-1).item()
            # 정답의 끝 위치와 관련된 로짓(outputs.end_logits)에서 가장 큰 값이 가리키는 토큰 위치
            end_pred = outputs.end_logits.argmax(dim=-1).item()
            # 정답 시작부터 끝까지의 토큰을 연결하여 정답 생성
            pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1])
    else:
        pred_text = ''
    return {
        'question': question,
        'context': context,
        'answer': pred_text
    }

# 검증

In [21]:
#from ratsnlp.nlpbook.qa import get_web_service_app
#app = get_web_service_app(inference_fn)
#app.run()

In [22]:
context = "한강대교(漢江大橋)는 서울특별시 용산구 이촌동에 있는 용산구 한강로3가와 동작구 본동 사이를 잇는 총연장 1,005m의 길이의 교량(다리)이다. 한강에 놓인 최초의 도로 교량으로, 제1한강교라고 불렸다. 1917년 개통된 뒤 몇 차례의 수난을 거쳐 지금에 이른다. 다리 아래로는 노들섬이 있다. 과거에는 국도 제1호선이 이 다리를 통하여 서울로 연결되었었다."
question = "한강대교 아래에는 어떤 섬이 있는가?"
inference_fn(question=question, context=context)

{'question': '한강대교 아래에는 어떤 섬이 있는가?',
 'context': '한강대교(漢江大橋)는 서울특별시 용산구 이촌동에 있는 용산구 한강로3가와 동작구 본동 사이를 잇는 총연장 1,005m의 길이의 교량(다리)이다. 한강에 놓인 최초의 도로 교량으로, 제1한강교라고 불렸다. 1917년 개통된 뒤 몇 차례의 수난을 거쳐 지금에 이른다. 다리 아래로는 노들섬이 있다. 과거에는 국도 제1호선이 이 다리를 통하여 서울로 연결되었었다.',
 'answer': '[UNK] [UNK] [UNK] ) 는 서울특별시 용산구 이촌동에 있는 용산구 한강로3가와 동작구 본동 사이를 잇는 총연장 1, 005m의 길이의 교량 ( 다리 ) 이다. 한강에 놓인 최초의 도로 교량으로, 제1한강교라고 불렸다. 1917년 개통된'}